In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import glob
import numpy as np
import plotly.plotly as py
import plotly.figure_factory as ff
from mpl_toolkits.basemap import Basemap
import folium
from folium.plugins import HeatMap
from folium import plugins
folium.plugins.HeatMap
from folium.plugins import MarkerCluster

%matplotlib inline

In [14]:
filelist = glob.glob('./Resources/*.csv')
filelist

#for filename in glob.glob('./Resources/*.csv')

['./Resources/April_2017.csv',
 './Resources/August_2017.csv',
 './Resources/December_2017.csv',
 './Resources/February_2017.csv',
 './Resources/January_2017.csv',
 './Resources/July_2017.csv',
 './Resources/June_2017.csv',
 './Resources/March_2017.csv',
 './Resources/May_2017.csv',
 './Resources/November_2017.csv',
 './Resources/October_2017.csv',
 './Resources/September_2017.csv']

In [15]:
f = open(filelist[0])
columns = f.readline()
columns = columns.strip().split(',')
columns[0] = 'X'
f.close()

columns += ['MONTH', 'SEASON']

master_df = pd.DataFrame(columns = columns)

for filename in filelist:
    month_csv = pd.read_csv(filename)
    month_df = pd.DataFrame(month_csv)
    month_df['MONTH'] = filename.split('/')[1][:-9]
    if "April" in filename or "March" in filename or "May" in filename:
        month_df["SEASON"] = "Spring"
    elif "January" in filename or "February" in filename or "December" in filename:
        month_df["SEASON"] = "Winter"
    elif "June" in filename or "July" in filename or "August" in filename:
        month_df["SEASON"] = "Summer"
    elif "September" in filename or "October" in filename or "November" in filename:
        month_df["SEASON"] = "Fall"
    else:
        month_df["SEASON"] = "Oh no!"
        
    master_df = pd.concat([master_df, month_df], axis=0, sort=False)

In [16]:
master_df = master_df.drop(["ROWID_", "DAY_OF_WEEK", "HOLIDAY", "WEEK_OF_YEAR", "MONTH_OF_YEAR", "ISSUE_TIME", "BODY_STYLE"], axis = 1)
master_df.head()

,X,Y,OBJECTID,VIOLATION_CODE,VIOLATION_DESCRIPTION,LOCATION,RP_PLATE_STATE,ADDRESS_ID,STREETSEGID,XCOORD,YCOORD,TICKET_ISSUE_DATE,MONTH,SEASON
0,-76.990896,38.892807,2750258,P050,PARKED WITHIN 25 FEET OF A STOP SIGN,1100 BLOCK PARK ST NE NORTH SIDE,VA,801357,1321.0,400790,136106,2017-04-24T08:36:00.000Z,,Spring
1,-76.980569,38.888851,2750259,P002,STAND OR PARK IN ALLEY,SIDE OF 1700 A ST SE SOUTH SIDE,DC,57985,13169.0,401686,135667,2017-04-24T10:26:00.000Z,,Spring
2,-76.980569,38.888851,2750260,P170,FAILURE TO DISPLAY CURRENT TAGS,SIDE OF 1700 A ST SE SOUTH SIDE,DC,57985,13169.0,401686,135667,2017-04-24T10:28:00.000Z,,Spring
3,-76.979301,38.888518,2750261,P303,RESERVED RESIDENTIAL SPACE PERSONS W/ DISABILTIES,FRONT OF 1753 A ST SE SOUTH SIDE,DC,56532,13169.0,401796,135630,2017-04-24T10:33:00.000Z,,Spring
4,-76.981491,38.888680,2750262,P003,RESIDENTIAL PERMIT PKING BEYOND LIMIT W/O PERMIT,1700 BLOCK A ST SE NORTH SIDE,VA,802308,2993.0,401606,135648,2017-04-24T12:23:00.000Z,,Spring


In [17]:
map_df = master_df[['X', 'Y']].copy()
map_df.head()

,X,Y
0,-76.990896,38.892807
1,-76.980569,38.888851
2,-76.980569,38.888851
3,-76.979301,38.888518
4,-76.981491,38.888680


In [18]:
map_df.count()

X    1404052
Y    1404052
dtype: int64

In [19]:
#take random sample from master dataframefor Folium handling
sampledata = map_df.take(np.random.permutation(len(map_df))[:1500])
sampledata.head()

,X,Y
81662,-76.930835,38.897678
21290,-77.036002,38.918124
5781,-77.062134,38.906817
36508,-76.976947,38.897237
79065,-77.019426,38.894797


In [8]:
m = folium.Map([38.9072, -77.0369], zoom_start=13)
m

In [9]:
#mark each ticketing location as a point to plot
for index, row in sampledata.iterrows():
    folium.CircleMarker([row['X'], row['Y']],
                        radius=15,
                        fill_color="#3db7e4",
                       ).add_to(m)

In [ ]:
sampledata["X"] = pd.to_numeric(sampledata["X"])
sampledata["Y"] = pd.to_numeric(sampledata["Y"])

ticketlocs = sampledata[['Y', 'X']].values.tolist()
ticketlocs

#plot heatmap
m.add_child(plugins.HeatMap(ticketlocs, radius=3))


m

In [11]:
mc = MarkerCluster()
for row in sampledata.itertuples():
    mc.add_child(folium.Marker(location=[row.Y, row.X]))
 
m.add_child(mc)
m

Had I had more weeks to dig into this data, I would have been interested to see how DC's ticketing rate stacks up against the rest of the country.

In [12]:
m2 = folium.Map([38.9072, -77.0369], zoom_start=13)
m2

some challenges faced: got far along with Folium, then learned they have a plotting limit. wanted to proportionally downsize the dataset by Address ID so that the volume of tickets by area remained the same, despite the overall volume of tickets in DC potentailly taking a hit, but didn't seem to be able to successfully plot more than ~1500, so chose to go with a random sample. 